In [155]:
import torch
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [156]:
# Read .csv
data = pd.read_csv('data.csv')
data.columns

Index([&#39;malignant&#39;, &#39;bilateral&#39;, &#39;age&#39;, &#39;month&#39;, &#39;bmi&#39;, &#39;diabetes&#39;,
       &#39;hypertension&#39;, &#39;surgery&#39;, &#39;smoking&#39;, &#39;family&#39;, &#39;menarche&#39;, &#39;menopause&#39;,
       &#39;miscarriage&#39;, &#39;pregnancy&#39;, &#39;childbearing&#39;, &#39;nd_blood&#39;, &#39;nd_milk&#39;,
       &#39;nd_serous&#39;, &#39;nd_change&#39;, &#39;nd_number&#39;, &#39;nd_mass&#39;, &#39;nd_position&#39;,
       &#39;nd_distance&#39;, &#39;nd_shape&#39;, &#39;nd_surface&#39;, &#39;nd_dispersion&#39;, &#39;nd_quantity&#39;,
       &#39;nd_distribution&#39;, &#39;nd_retraction&#39;, &#39;nd_appearance&#39;, &#39;us_ductectasia&#39;,
       &#39;us_intramass&#39;, &#39;us_cysmass&#39;, &#39;us_mass&#39;, &#39;us_size&#39;, &#39;us_position&#39;,
       &#39;us_distance&#39;, &#39;us_shape&#39;, &#39;us_boundary&#39;, &#39;us_bloodflow&#39;,
       &#39;us_calcification&#39;, &#39;us_birads&#39;, &#39;xray&#39;],
      dtype=&#39

In [157]:
# 指定标签及特征

label = data['malignant']
features = data.drop(columns=['malignant'])

In [158]:
# One-Hot 编码分类数据

features = pd.get_dummies(features)

In [159]:
# 数值标准化

features = preprocessing.StandardScaler().fit_transform(features)

In [160]:
# 指定 x, y

x = torch.from_numpy(np.array(features)).float().cuda()
y = torch.from_numpy(np.array(label)).cuda()

In [161]:
x.shape

torch.Size([355, 42])

In [162]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=12)

In [163]:
# 构建网络

net = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 1028),
    torch.nn.ReLU(),
    torch.nn.Linear(1028, 2)
)

optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
loss_func = torch.nn.CrossEntropyLoss()

In [164]:
net = net.cuda()

In [165]:
for t in range(500):
    prediction = net(x_train)
    loss = loss_func(prediction, y_train)
    print(t, loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

evice=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
241 tensor(0.1089, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
242 tensor(0.1084, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
243 tensor(0.1080, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
244 tensor(0.1075, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
245 tensor(0.1071, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
246 tensor(0.1066, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
247 tensor(0.1062, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
248 tensor(0.1058, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
249 tensor(0.1053, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
250 tensor(0.1049, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
251 tensor(0.1045, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
252 tensor(0.1041, device=&#39;cuda:0&#39;, grad_fn=&lt;NllLossBackward&gt;)
253 tensor(0.1036, 

In [166]:
y_predict = torch.max(torch.nn.functional.softmax(net(x_test)),1)[1]

In [167]:
y_predict

tensor([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0], device=&#39;cuda:0&#39;)

In [168]:
maxtrix = confusion_matrix(y_test.cpu(), y_predict.cpu())

In [169]:
maxtrix

array([[65, 10],
       [11, 21]], dtype=int64)